In [1]:
from ch2.data import data
from ch2.lib.date import to_time
d = data()
g = d.activity_groups()
print(g)
bike = g.loc[g.name == 'Bike'].iloc[0].id
print(bike)
a = d.activity_journals('Bike')
#x = a.loc[(a.index >= to_time('2017-01-18')) & (a.index < to_time('2017-01-19'))]
x = a.loc[(a.index >= to_time('2017-07-01')) & (a.index < to_time('2017-07-02'))]
print(x)
ride = d.statistic_journals('Spherical Mercator X', 'Spherical Mercator Y', 'Latitude', 'Longitude', 'Distance', source_ids=x.source_id)
print(ride)

    INFO: Using database at /home/andrew/.ch2/database.sqln


   name             description  id  count
1  Bike  All cycling activities   1      0
2   Run  All running activities   2      0
1
Empty DataFrame
Columns: [source_id]
Index: []
Empty DataFrame
Columns: []
Index: []


In [2]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.tile_providers import CARTODBPOSITRON
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models.widgets import Slider, PreText

output_notebook()
width, height = 800, 800

def modify_doc(doc):

    t1 = PreText(height=20, width=200)
    t2 = PreText(height=20, width=200)
    t3 = PreText(height=20, width=100)
    s1 = Slider(start=0, end=len(ride), value=0, title='Start')
    s2 = Slider(start=1, end=len(ride)-1, value=len(ride), title='Length')
    f = figure(plot_width=width, plot_height=height, x_axis_type='mercator', y_axis_type='mercator')
    c = column(row(s1, s2), row(t1, t2, t3), f)

    def mkplot(l1, l2):
        l2 = min(len(ride)-1, l1+l2)
        t1.text = '%9.5f,%9.5f' % (ride.iloc[l1]['Longitude'], ride.iloc[l1]['Latitude'])
        t2.text = '%9.5f,%9.5f' % (ride.iloc[l2]['Longitude'], ride.iloc[l2]['Latitude'])
        t3.text = '%4.1fkm' % ((ride.iloc[l2]['Distance'] - ride.iloc[l1]['Distance']) / 1000)
        f = figure(plot_width=width, plot_height=height, x_axis_type='mercator', y_axis_type='mercator')
        f.circle(x='Spherical Mercator X', y='Spherical Mercator Y', source=ride[l1:l2])
        f.add_tile(CARTODBPOSITRON)
        c.children[2] = f

    s1.on_change('value', lambda attr, old, new: mkplot(s1.value, s2.value))
    s2.on_change('value', lambda attr, old, new: mkplot(s1.value, s2.value))
    doc.add_root(c)

show(modify_doc)


Loading BokehJS ...

In [2]:
from ch2.squeal.database import connect
from ch2.squeal.tables.segment import Segment
ns, log, db = connect(['-v', '5'])
with db.session_context() as s:
    x = Segment(start=(-70.61813,-33.41536), finish=(-70.63340,-33.42655), distance=4400,
                activity_group_id=bike,
                name='San Cristobal', description='Climb up San Cristobal in Parque Metropolitana')
    s.add(x)

    INFO: Using database at /home/andrew/.ch2/database.sqln


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: 'INSERT INTO segment (activity_group_id, start_lat, start_lon, finish_lat, finish_lon, distance, name, description) VALUES (?, ?, ?, ?, ?, ?, ?, ?)'] [parameters: (1, -33.41536, -70.61813, -33.42655, -70.6334, 4400.0, 'San Cristobal', 'Climb up San Cristobal in Parque Metropolitana')] (Background on this error at: http://sqlalche.me/e/gkpj)